In [2]:
import pandas as pd
import glob
import re
from pandas import json_normalize
import json

from pyspark.sql import SparkSession
from pyspark import  SparkConf

# Set up pyspark session

In [3]:
appName = "app"
master = "local[*]" # Spark will use all cores (*) available
if not 'spark' in globals(): # This 'trick' makes sure the SparkContext sc is initialized exactly once
  conf = SparkConf().setAppName(appName).setMaster(master)
  spark = SparkSession.builder \
        .config(conf=conf) \
        .getOrCreate()

24/06/05 10:57:26 WARN Utils: Your hostname, Jonnys-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.60.170.206 instead (on interface en0)
24/06/05 10:57:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/05 10:57:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/05 10:57:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
#Display the information kept in the spark variable
spark

# Load and explore raw data
## Cultural sites

In [16]:
cultural_cites.columns

['addresses_roadtype_name',
 'addresses_end_street_number',
 'values_attribute_name',
 'addresses_road_name',
 'values_category',
 'addresses_zip_code',
 'values_value',
 'addresses_town',
 'geo_epgs_4326_y',
 'geo_epgs_4326_x',
 'addresses_district_name',
 'geo_epgs_25831_x',
 'addresses_start_street_number',
 'register_id',
 'institution_id',
 'addresses_main_address',
 'addresses_district_id',
 'addresses_roadtype_id',
 'addresses_type',
 'addresses_neighborhood_id',
 '_id',
 'name',
 'addresses_road_id',
 'created',
 'geo_epgs_25831_y',
 'institution_name',
 'modified',
 'values_description',
 'values_id',
 'addresses_neighborhood_name',
 'values_outstanding',
 'values_attribute_id']

In [5]:
cultural_cites_df = pd.read_csv('../datasets/cultural-sites/2023-04-20.csv')
cultural_cites_df

,addresses_roadtype_name,addresses_end_street_number,values_attribute_name,addresses_road_name,values_category,addresses_zip_code,values_value,addresses_town,geo_epgs_4326_y,geo_epgs_4326_x,...,addresses_road_id,created,geo_epgs_25831_y,institution_name,modified,values_description,values_id,addresses_neighborhood_name,values_outstanding,values_attribute_id
0,NaN,NaN,E-mail,C Esteve Terradas,Informació d'interès,8023,pvirgili@perevirgili.catsalut.net,BARCELONA,2.140266,41.414955,...,116901,1988-03-14T00:00:00,4.585179e+06,NaN,2022-09-21T19:13:34.303475,NaN,40212.0,Vallcarca i els Penitents,True,100002.0
1,NaN,NaN,Web,C Fulton,Informació d'interès,8032,http://lameva.barcelona.cat/horta-guinardo/ca/...,BARCELONA,2.161502,41.429994,...,137209,2015-05-26T12:24:46,4.586832e+06,NaN,2021-09-09T02:14:37.921873,NaN,95717.0,Horta,True,100003.0
2,NaN,NaN,NaN,Pl Lesseps,NaN,8023,NaN,BARCELONA,2.151087,41.407712,...,123408,2015-05-26T13:26:20,4.584366e+06,NaN,2021-09-09T02:14:44.349060,NaN,NaN,la Vila de Gràcia,NaN,NaN
3,NaN,NaN,E-mail,C Ramon Trias Fargas,Informació d'interès,8005,info@idec.upf.edu,BARCELONA,2.190872,41.390029,...,132908,2015-05-26T17:06:23,4.582371e+06,NaN,2022-09-09T12:06:57.108133,NaN,57030.0,la Vila Olímpica del Poblenou,True,100002.0
4,NaN,NaN,Web,Pg Vall d'Hebron,Informació d'interès,8035,http://www.ub.edu/campusmundet/,BARCELONA,2.147158,41.435755,...,352507,2015-05-26T17:35:12,4.587483e+06,NaN,2022-09-09T12:07:54.518743,NaN,95719.0,Montbau,True,100003.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866,NaN,NaN,E-mail,C Maspons,Informació d'interès,8012,administracio@lavioleta.cat,BARCELONA,2.156215,41.401031,...,203100,2007-01-15T14:32:20,4.583620e+06,NaN,2022-09-17T06:45:19.064305,NaN,52042.0,la Vila de Gràcia,True,100002.0
867,NaN,NaN,Web,Pg Joan Borbó C Barcelona,Informació d'interès,8039,http://www.telefericodebarcelona.com/ca/,BARCELONA,2.187876,41.373250,...,224904,2008-04-22T14:37:02,4.580511e+06,NaN,2021-04-22T14:26:50.786623,NaN,88784.0,la Barceloneta,True,100003.0
868,NaN,NaN,E-mail,Carrer de Sant Pere Més Baix,Informació d'interès,8003,casaldebarripoudelafiguera@gmail.com,Barcelona,2.178657,41.388199,...,31707,2010-10-25T12:46:13,4.582178e+06,NaN,2022-10-04T13:22:18.225934,NaN,54899.0,"Sant Pere, Santa Caterina i la Ribera",True,100002.0
869,NaN,NaN,E-mail,Pl Rosa dels Vents,Informació d'interès,8039,wbarcelona@whotels.com,BARCELONA,2.189998,41.368361,...,701643,2009-07-29T10:33:54,4.579966e+06,NaN,2022-09-17T07:14:02.084947,NaN,53902.0,la Barceloneta,True,100002.0


In [6]:
# Set File Path
cultural_citesFilePath = '../datasets/cultural-sites/2023-04-20.csv' 

# Obtain Departure Delays dataset
cultural_cites = spark.read.csv(cultural_citesFilePath, header='true', inferSchema='true').cache()
print("File loaded!!!")

File loaded!!!


24/06/05 10:57:29 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [7]:
cultural_cites.show(5)

+-----------------------+---------------------------+---------------------+--------------------+--------------------+------------------+--------------------+--------------+------------------+------------------+-----------------------+------------------+-----------------------------+------------+--------------+----------------------+---------------------+---------------------+--------------+-------------------------+---+--------------------+-----------------+-------------------+------------------+----------------+--------------------+------------------+---------+---------------------------+------------------+-------------------+
|addresses_roadtype_name|addresses_end_street_number|values_attribute_name| addresses_road_name|     values_category|addresses_zip_code|        values_value|addresses_town|   geo_epgs_4326_y|   geo_epgs_4326_x|addresses_district_name|  geo_epgs_25831_x|addresses_start_street_number| register_id|institution_id|addresses_main_address|addresses_district_id|addresse

## Idealista

In [8]:
json_files = glob.glob('../datasets/idealista/*.json')
idealista_df = []

for file in json_files:

    # Extract date from filename 
    date_match = re.search(r'(\d{4}_\d{2}_\d{2})', file)
    
    # Extract the matched date string
    date_str = date_match.group(1)  

    # Replace underscores with hyphens 
    formatted_date = date_str.replace('_', '-')
    
    # Load JSON file
    idealista_temp = pd.read_json(file)
    
    # Add a new column with the extracted date
    idealista_temp['date'] = formatted_date
    
    # Append the DataFrame with the date to the list
    idealista_df.append(idealista_temp)

# Concatenate all DataFrames in the list into one DataFrame
idealista_df = pd.concat(idealista_df, ignore_index=True)
idealista_df

,propertyCode,thumbnail,externalReference,numPhotos,floor,price,propertyType,operation,size,exterior,...,detailedType,suggestedTexts,hasPlan,has3DTour,has360,hasStaging,topNewDevelopment,parkingSpace,date,newDevelopmentFinished
0,88841013.0,https://img3.idealista.com/blur/WEB_LISTING/0/...,W-02I842,22.0,8,293000.0,flat,sale,80.0,False,...,{'typology': 'flat'},"{'subtitle': 'La Vall d'Hebron - La Clota, Bar...",True,False,True,False,False,NaN,2020-08-06,NaN
1,93199277.0,https://img3.idealista.com/blur/WEB_LISTING/0/...,NaN,24.0,2,166000.0,flat,sale,63.0,True,...,{'typology': 'flat'},"{'subtitle': 'La Prosperitat, Barcelona', 'tit...",True,False,False,False,False,NaN,2020-08-06,NaN
2,93198927.0,https://img3.idealista.com/blur/WEB_LISTING/0/...,NaN,5.0,st,39000.0,flat,sale,50.0,False,...,{'typology': 'flat'},"{'subtitle': 'La Trinitat Vella, Barcelona', '...",True,False,False,False,False,NaN,2020-08-06,NaN
3,93155700.0,https://img3.idealista.com/blur/WEB_LISTING/0/...,D-40702,27.0,4,190000.0,flat,sale,82.0,True,...,{'typology': 'flat'},{'subtitle': 'Can Peguera - El Turó de la Peir...,True,False,False,False,False,NaN,2020-08-06,NaN
4,92704898.0,https://img3.idealista.com/blur/WEB_LISTING/0/...,3786,37.0,1,199000.0,flat,sale,71.0,True,...,{'typology': 'flat'},"{'subtitle': 'Porta, Barcelona', 'title': 'Pis...",True,False,False,False,False,NaN,2020-08-06,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21068,89232336.0,https://img3.idealista.com/blur/WEB_LISTING/0/...,1104681_V,9.0,en,250000.0,flat,sale,96.0,True,...,{'typology': 'flat'},"{'subtitle': 'Centre, Esplugues de Llobregat',...",False,False,False,False,False,NaN,2020-05-12,NaN
21069,91302966.0,https://img3.idealista.com/blur/WEB_LISTING/0/...,27754,30.0,NaN,1380000.0,chalet,sale,519.0,False,...,{'typology': 'chalet'},"{'subtitle': 'Centre, Sant Just Desvern', 'tit...",False,False,False,False,False,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",2020-05-12,NaN
21070,84482895.0,https://img3.idealista.com/blur/WEB_LISTING/0/...,25285,42.0,NaN,890000.0,chalet,sale,310.0,False,...,{'typology': 'chalet'},"{'subtitle': 'Centre, Sant Just Desvern', 'tit...",False,False,False,False,False,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",2020-05-12,NaN
21071,92399305.0,https://img3.idealista.com/blur/WEB_LISTING/0/...,V1650,19.0,3,87000.0,flat,sale,55.0,False,...,{'typology': 'flat'},"{'subtitle': 'La Florida, Hospitalet de Llobre...",False,False,False,False,False,NaN,2020-05-12,NaN


## Incidences

In [9]:
csv_files = glob.glob('../datasets/incidences/*.csv')
incidences_df = []

for file in csv_files:

    # Extract date from filename 
    year_match = re.search(r'(\d{4})', file)
    
    # Extract the matched year string
    year_str = year_match.group(1)  

    # convert the string to an integer
    year = int(year_str)

    # Load file
    incidences_temp = pd.read_csv(file)
    
    # Add a new column with the extracted date
    incidences_temp['YEAR'] = year
    
    # Append the DataFrame with the date to the list
    incidences_df.append(incidences_temp)

# Concatenate all DataFrames in the list into one DataFrame
incidences_df = pd.concat(incidences_df, ignore_index=True)
incidences_df

,CODI_DISTRICTE,DISTRICTE,ANY_DATA_TANCAMENT,DETALL,BARRI,FITXA_ID,LATITUD,AREA,COORDENADA_Y,COORDENADA_X,...,TIPUS_VIA,ANY_DATA_ALTA,MES_DATA_ALTA,SUPORT,CARRER,CODI_BARRI,_id,TIPUS,DIA_DATA_TANCAMENT,YEAR
0,3.0,Sants-Montjuïc,2016,Disconforme servei de neteja carrers / places,la Marina de Port,249606,41.362124,Recollida i neteja de l'espai urbà,4579312.661,428231.1953,...,Carrer,2015,12,TELÈFON,Foneria,13.0,1,QUEIXA,1,2016
1,5.0,Sarrià-Sant Gervasi,2016,Disconforme servei de neteja carrers / places,les Tres Torres,249604,41.401790,Recollida i neteja de l'espai urbà,4583727.945,427120.2991,...,Carrer,2015,12,WEB,Calatrava,24.0,2,QUEIXA,1,2016
2,2.0,Eixample,2016,"Orins, excrements animals",Sant Antoni,249615,41.375867,Recollida i neteja de l'espai urbà,4580821.582,429969.0695,...,Carrer,2015,12,TELÈFON,Viladomat,10.0,3,INCIDENCIA,1,2016
3,2.0,Eixample,2016,Disconforme servei de neteja carrers / places,l'Antiga Esquerra de l'Eixample,249616,41.390159,Recollida i neteja de l'espai urbà,4582412.364,429564.1159,...,Carrer,2015,12,TELÈFON,Mallorca,8.0,4,QUEIXA,1,2016
4,7.0,Horta-Guinardó,2016,Disconforme servei de neteja carrers / places,el Guinardó,249617,41.419610,Recollida i neteja de l'espai urbà,4585666.666,431179.5351,...,Ronda,2015,12,WEB,Guinardó,35.0,5,QUEIXA,1,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,NaN,NaN,2014,Rec. mobles i trastos vells gratuïta gent gran...,NaN,68407,NaN,Gestions municipals,NaN,NaN,...,NaN,2014,8,TELÈFON,NaN,NaN,95,PETICIO DE SERVEI,13,2014
896,NaN,NaN,2014,Rec. mobles i trastos vells gratuïta gent gran...,NaN,68408,NaN,Gestions municipals,NaN,NaN,...,NaN,2014,8,TELÈFON,NaN,NaN,96,PETICIO DE SERVEI,13,2014
897,2.0,Eixample,2014,Contenidor escombraries incidències,l'Antiga Esquerra de l'Eixample,68400,41.392153,Recollida i neteja de l'espai urbà,4582632.705,429664.6977,...,Carrer,2014,8,TELÈFON,Balmes,8.0,97,INCIDENCIA,13,2014
898,5.0,Sarrià-Sant Gervasi,2014,Parcs jardins i zones verdes fuites aigua,Sant Gervasi - la Bonanova,68401,41.407882,Manteniment de l'espai urbà,4584396.446,427898.2024,...,Plaça,2014,8,TELÈFON,Rosa Sabater,25.0,98,INCIDENCIA,13,2014


## Income

In [10]:
csv_files = glob.glob('../datasets/income/*.csv')
income_df = []

for file in csv_files:

    # Extract date from filename 
    year_match = re.search(r'(\d{4})', file)
    
    # Extract the matched year string
    year_str = year_match.group(1)  

    # convert the string to an integer
    year = int(year_str)

    # Load file
    income_temp = pd.read_csv(file)
    
    # Add a new column with the extracted date
    income_temp['YEAR'] = year
    
    # Append the DataFrame with the date to the list
    income_df.append(income_temp)

# Concatenate all DataFrames in the list into one DataFrame
income_df = pd.concat(income_df, ignore_index=True)

# Check if the 'YEAR' column is equal to the 'Any' column
income_df['YEAR'].eq(income_df['Any']).value_counts()

# Drop the 'Any' column
income_df.drop(columns=['Any'], inplace=True)

income_df

,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Població,Índex RFD Barcelona = 100,YEAR
0,1,Ciutat Vella,1,el Raval,48485,65.0,2011
1,1,Ciutat Vella,2,el Barri Gòtic,17257,98.8,2011
2,1,Ciutat Vella,3,la Barceloneta,15674,73.1,2011
3,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",22632,86.1,2011
4,2,Eixample,5,el Fort Pienc,32348,97.9,2011
...,...,...,...,...,...,...,...
806,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,9921,104.4,2008
807,10,Sant Martí,70,el Besòs i el Maresme,23557,61.1,2008
808,10,Sant Martí,71,Provençals del Poblenou,18556,85.1,2008
809,10,Sant Martí,72,Sant Martí de Provençals,26199,80.3,2008


## Price open data

In [11]:
with open("../datasets/price_opendata/price_opendata_neighborhood.json", 'r') as file:
    # Load all JSON objects into a list
    prices_temp  = [json.loads(line) for line in file]

prices_list = []

for entry in prices_temp :
    # Normalize the nested 'info' part of each entry
    prices_temp_df = json_normalize(entry, record_path='info', meta=['_id', 'neigh_name', 'district_id', 'district_name'], errors='ignore')
    prices_list.append(prices_temp_df)

# Concatenate all DataFrames from the list into one DataFrame
prices_df = pd.concat(prices_list, ignore_index=True)

prices_df

,year,Amount,usedAmount,PerMeter,usedPerMeter,diffAmount,diffPerMeter,_id,neigh_name,district_id,district_name
0,2013,97.0,97.0,1726.5,1726.5,NaN,NaN,1,NaN,1,Ciutat Vella
1,2014,141.7,143.1,2087.6,2073.5,99.1,2534.3,1,NaN,1,Ciutat Vella
2,2015,193.8,193.8,2401.9,2401.9,NaN,NaN,1,NaN,1,Ciutat Vella
3,2016,181.0,180.7,2805.2,2798.6,NaN,NaN,1,NaN,1,Ciutat Vella
4,2017,240.3,240.0,3469.9,3468.9,292.5,3633.1,1,NaN,1,Ciutat Vella
...,...,...,...,...,...,...,...,...,...,...,...
354,2016,125.8,126.8,1790.6,1770.9,116.6,1972.2,72,NaN,10,Sant Martí
355,2017,138.3,138.8,2065.7,2026.4,134.2,2417.2,72,NaN,10,Sant Martí
356,2014,55.9,55.9,684.5,684.5,NaN,NaN,73,NaN,3,Sants-Montjuïc
357,2016,79.3,79.3,1088.7,1088.7,NaN,NaN,73,NaN,3,Sants-Montjuïc


## Unemployment

In [12]:
json_files = glob.glob('../datasets/unemployment/*.json')
unemployment_df = []

for file in json_files:

    # Extract date from filename 
    year_match = re.search(r'(\d{4})', file)
    
    # Extract the matched date string
    year_str = year_match.group(1)  

    # convert the string to an integer
    year = int(year_str)
    
    # Load JSON file
    unemployment_temp = pd.read_json(file)
    
    # Add a new column with the extracted year
    unemployment_temp['year'] = year
    
    # Append the DataFrame with the date to the list
    unemployment_df.append(unemployment_temp)

# Concatenate all DataFrames in the list into one DataFrame
unemployment_df = pd.concat(unemployment_df, ignore_index=True)
unemployment_df

,help,success,result,year,error
0,https://opendata-ajuntament.barcelona.cat/data...,True,0f1ed6a5-1224-4dec-94ee-92b542ad3c20,2019,NaN
1,https://opendata-ajuntament.barcelona.cat/data...,True,"[{'type': 'int4', 'id': '_id'}, {'type': 'nume...",2019,NaN
2,https://opendata-ajuntament.barcelona.cat/data...,True,"[{'Codi_Districte': '1', 'Sexe': 'Homes', 'Nom...",2019,NaN
3,https://opendata-ajuntament.barcelona.cat/data...,True,{'start': '/api/action/datastore_search?resour...,2019,NaN
4,https://opendata-ajuntament.barcelona.cat/data...,True,3552,2019,NaN
...,...,...,...,...,...
57,https://opendata-ajuntament.barcelona.cat/data...,True,"[{'Codi_Districte': '1', 'Sexe': 'Homes', 'Nom...",2018,NaN
58,https://opendata-ajuntament.barcelona.cat/data...,True,{'start': '/api/action/datastore_search?resour...,2018,NaN
59,https://opendata-ajuntament.barcelona.cat/data...,True,3552,2018,NaN
60,https://opendata-ajuntament.barcelona.cat/data...,False,NaN,2015,"Not found: Resource ""f79e8446-11d6-44d7-bbb2-f..."
